In [ ]:
import os
job_class = os.getenv("DKUBE_JOB_CLASS")
if not job_class:
    !{sys.executable} -m pip install kfp >/dev/null

In [ ]:
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

In [ ]:
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
image = "ocdr/dkube-datascience-tf-cpu:v2.0.0-16"
serving_image = "ocdr/tensorflowserver:2.0.0"
# training_program = 'HOMEDIR/workspace/training'
training_program = "training"
model = 'insurance'
training_script = "python insurance/training.py"
transformer_code='insurance/transformer.py'
user = os.getenv('USERNAME')
framework = "tensorflow"
f_version = "2.0.0"
output_mount_point = "/opt/dkube/out"

## Project ID, make sure IDE is created under the same project where the expriment is suppose to run
## Or else provide it manually in the project_id variable.
project_id = os.environ.get("DKUBE_PROJECT_ID")
deployment_name  = os.getenv("DKUBE_PROJECT_NAME", "insurance") + "-deployment"

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-insurance-pl',
    description='sample insurance pipeline'
)
def insurance_pipeline(token):
    
    train       = dkube_training_op(token, json.dumps({"image": image}),
                                    tags=json.dumps([f"project:{project_id}"]),
                                    framework=framework, version=f_version,
                                    program=training_program, run_script=training_script,
                                    outputs=json.dumps([model]),
                                    output_mounts=json.dumps([output_mount_point]))
    
    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    name=deployment_name,
                                    serving_image=json.dumps({"image": serving_image}),
                                    transformer_image=json.dumps({"image": image}),
                                    transformer_project=training_program,
                                    transformer_code=transformer_code,
                                    production="true").after(train)

## Create a run

In [ ]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
client.create_run_from_pipeline_func(insurance_pipeline, arguments={"token":existing_token}, experiment_name="default")